In [82]:
from urllib import parse
import pandas as pd
import numpy as np
import lxml
import tensorflow as tf

In [83]:
def l2Norm(inp):# 환율데이터 표본적어서 *10
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp))*10)
    return result
def l3Norm(inp):# *10안함
    result = []
    for i in inp:
        result.append(((i-np.mean(inp))/np.var(inp)))
    return result

class Stock:
    

    
    Exchange='./Exchange.csv' #객체가 공통으로 사용할 환율 데이터 (동적크롤링할줄몰라서 csv로 받아옴)
    
    def __init__(self,code):
        self.code=code
    
    @staticmethod
    def exchangeRate():  ##1달간격의 환율의 종가데이터  1행배열 57요소( 18.05.01~  23.01.01) 배열로 출력 SIZE(1,57)
        csv = pd.read_csv(Stock.Exchange,encoding='utf-8')
        #print(csv)  환율 전체데이터 확인하기            
        csv = np.array(csv)
        data=[]
        for i in range(len(csv)):   
            fnum=""
            for j in csv[i][1]:
                if(j == ','):
                    continue
                fnum+=j
            data.append(float(fnum))
        data = np.array(data)
        #정규화
        data = data.astype('float32')
        data = l2Norm(data) 
        return data

    def idxNorm(self):##ROA, ROE, EPS, BPS, DPS, PER, PBR의 각 항목당 (2018.12 ~ 2023.12)의 데이터 6개 갖음 SIZE(7,8)
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        sit=np.array(tables[11]) 
        
        data=[]#정규화 시작
        idx=np.array([])
        for i in range(7):
            idx = sit[17+i][2:8].astype('float32')
            idx =l3Norm(idx)
            data.append(idx)
        data=np.array(data)
        return data
    
    #def priceList(self): ## 주식 종가를 데이터로 받아옴
    
    ##연비 구현바람
    
    
    #return 


    def showChart(self): ##종목코드의 제무재표를 나타내는 함수
        get_param = {
            'pGB':1,
            'gicode':'A%s'%(self.code),
            'cID':'',
            'MenuYn':'Y',
            'ReportGB':'',
            'NewMenuID':101,
            'stkGb':701,
        }
        get_param = parse.urlencode(get_param)
        url="http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?%s"%(get_param)
        tables = pd.read_html(url, header=0,encoding='utf-8')
        return tables[11]
    

In [84]:
#SS = Stock('005930')
#SK = Stock('000660')
#DW=Stock('049770')
SFA=Stock('056190')
#GW=Stock('001570')

print('정규화 환율\n',Stock.exchangeRate(),'\n') #환율 나타내기
print('정규화 지표값\n',SFA.idxNorm(),'\n') #지표값 정규화
SFA.showChart() #삼성재무재표 바로보기

정규화 환율
 [0.15273633413016796, 0.13651004992425442, 0.2095378376543522, 0.4321892186999321, 0.457523949444294, 0.2804318815469742, 0.21190447732806206, 0.18418516963720322, 0.10305309668183327, 0.1402932032942772, 0.05268852226436138, 0.027497154660522938, 0.03558348398655653, 0.005694530555047095, -0.0037185821565799415, -0.018492669332772493, -0.0009932270040735602, -0.04671406000852585, -0.05983853247016668, -0.09736557491123676, -0.14021769165992737, -0.12124788947403431, -0.10330020450055599, -0.10918120853602886, -0.12038729153573513, -0.17986031249165535, -0.13530495576560497, -0.08658978156745434, -0.0360995763912797, 0.005658636218868196, 0.015788981691002846, 0.02599111758172512, 0.08374273777008057, 0.0504829827696085, 0.06009351462125778, 0.0276405131444335, 0.018263295060023665, -0.055069378577172756, -0.007322477176785469, -0.027869888581335545, 0.023857364431023598, 0.04431525710970163, 0.004547000862658024, -0.05272048059850931, 0.005407812423072755, -0.03860978409647941

,IFRS(연결),Annual,Annual.1,Annual.2,Annual.3,Annual.4,Annual.5,Annual.6,Annual.7
0,IFRS(연결),2017/12,2018/12,2019/12,2020/12,2021/12,2022/12(E),2023/12(E),2024/12(E)
1,매출액,19204,15601,15777,15512,15649,17178,19912,22165
2,영업이익,2361,2344,2142,1673,1889,1978,2368,2679
3,영업이익(발표기준),2361,2344,2142,1673,1889,NaN,NaN,NaN
4,당기순이익,2033,1697,1470,1186,1483,1315,1861,2143
5,지배주주순이익,1969,1579,1320,1083,1169,1067,1507,1729
6,비지배주주순이익,64,117,150,104,315,NaN,NaN,NaN
7,자산총계,18108,16605,18059,17994,19044,20059,21989,24255
8,부채총계,8260,6575,6938,5199,5041,5253,5644,6123
9,자본총계,9849,10030,11121,12795,14004,14805,16345,18132
